# Enunciado
## Destino: Planeta VASS!
Por fin! El sistema que hemos creado ha terminado de analizar las características del planeta y se ha comprobado que es habitable. Después de un largo trayecto y varios inconvenientes, estamos a un paso de cumplir nuestro objetivo!

Este último paso es, probablemente, el más importante de todos, aterrizar en el planeta VASS.
Para asegurar el éxito del aterrizaje, hemos realizado una serie de experimentos en un espacio de simulación para predecir la manera más eficiente de aterrizar de forma segura.

Sin embargo, el simulador ha tenido problemas a la hora de determinar la eficiencia en algunos de los experimentos, por lo que ahora nuestro objetivo será analizar las simulaciones y determinar el nivel de eficiencia de aterrizaje de cada uno de los experimentos en los que falló. Tendremos que tener en cuenta las siguientes observaciones:

### Observaciones
- Cada uno de los experimentos cuenta con una serie pasos compuestos de estados y acciones de la nave de una simulación determinada.
- En cada paso el simulador genera un estado y decide una acción. El siguiente estado se verá determinado por esta acción, y a su vez se decidirá una nueva acción en base a ese nuevo estado, eso hasta que la simulación llegue a su fin.
- En cada paso, el simulador determina la contribución individual que ha tenido la acción tomada a la eficiencia total. Finalmente se calculan todas estas contribuciones para determinar la eficiencia total del experimento.
- La eficiencia es una función que utiliza internamente el simulador para determinar cuándo está tomando buenas decisiones. Esta función podría estar teniendo en cuenta los cambios que hay en los estados, así como las distintas acciones y variables.
- Las simulaciones se han realizado en diversas condiciones: gravedad, viento y turbulencias.
- CUIDADO! En algunas de las simulaciones los valores de la gravedad y viento no se guardaron correctamente, por lo que el sistema las ha registrado con un valor de 0.

Descarga los datos de los experimentos, analízalos y crea un modelo para predecir la eficiencia total de cada uno de los experimentos. Por último, observa los experimentos del conjunto de test que viene incluido en la descarga de los datos y envía las predicciones de los registros en el orden original separadas por comas, de la siguiente manera:

205.12,122.14,80.34,30.15,109.93

El ejemplo expuesto se correspondería con las predicciones de los 5 primeros registros, tendréis que subir en una sola línea las 2,000 predicciones usando este formato.

# How to land the Star Cruiser 42

## Context

This dataset contains information about simulations of landing a spaceship in a 2D environment.
This information can be used to determine the efficiency function in order to evaluate the best actions possible in a future landing.


## Metadata
Number of train files: 9513
Number of test files: 2000
Number of variables: 13
Target variable: efficiency


File variable details:

- x_pos # The position of the ship on the X axis, in units relative to the landing platform.
- y_pos # The position of the ship on the Y axis, in units relative to the landing platform.
- x_vel # Velocity of the ship on the X axis.
- y_vel # Velocity of the ship on the Y axis.
- angle # The angle of the spaceship in radians.
- ang_vel # The angular velocity of the spaceship.
- leg_1 # Boolean that represent whether leg 1 is in contact with the ground or not
- leg_2 # Boolean that represent whether leg 2 is in contact with the ground or not
- main_booster #  The throttle of the main engine.
- lat_booster # The throttle of the lateral boosters.

The first 6 variables determine the state and each step, and the last two (main_booster and lat_booster) are the actions taken in response
to that state, whose repercussions will be seen in the following step.

Experiment variable details:
- gravity # The magnitude of the gravity acceleration. A value of 0 means that the system didn't register the real value.
- wind_power # The maximum magnitude of linear wind applied to the spaceship. A value of 0 means that the system didn't register the real value.
- turbulences # The maximum magnitude of rotational wind applied to the spaceship.

Target variable detail
- efficiency # An internal function of the simulator to evaluate the quality of the landing.
    

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew

import matplotlib.pyplot as plt
import seaborn as sns

import os, os.path
import errno

In [ ]:
# Open fase7/data/experiments_summary_test.xlsx and train
experiments_summary_test = pd.read_excel('data/experiments_summary_test.xlsx')
experiments_summary_train = pd.read_excel('data/experiments_summary_train.xlsx')

In [ ]:
experiments_summary_train

In [ ]:
experiments_summary_test

In [ ]:
experiments_summary_train.describe()

In [ ]:
experiments_summary_test.describe()

In [ ]:
experiments_summary = pd.concat([experiments_summary_train, experiments_summary_test], ignore_index=True)
experiments_summary.describe()

Los datos con los que vamos a trabajar son experimentos que se componen por series temporales de longitud variable. Para un experimento i, contamos con un conjunto de datos de la forma (t_j, x_j), donde t_j es el timestamp y x_j el vector de variables.

Cada experimento i cuenta con:
- gravity
- wind_power
- tubulence_power
- eficciency

El vector de variables x_j está compuesto por las variables:
- x_pos
- y_pos
- x_vel
- y_vel
- angle
- ang_vel
- leg1
- leg2
- main_booster
- lat_booster


In [ ]:
for col in experiments_summary_train.columns[1:]:
    fig = plt.figure(figsize=(9, 3))
    sns.histplot(data=experiments_summary_train, x=col, kde=True)
    plt.show()
    fig.clf()

    print('Skewness:', skew(experiments_summary_train[col]))
    print('Kurtosis:', kurtosis(experiments_summary_train[col]))

In [ ]:
for col in experiments_summary_test.columns[1:]:
    fig = plt.figure(figsize=(9, 3))
    sns.histplot(data=experiments_summary_test, x=col, kde=True)
    plt.show()
    fig.clf()

    print('Skewness:', skew(experiments_summary_test[col]))
    print('Kurtosis:', kurtosis(experiments_summary_test[col]))

In [ ]:
for q in range(100):
    if np.percentile(experiments_summary_train['efficiency'], q) >= 0:
        print(q)
        break

In [ ]:
fig = plt.figure(figsize=(9, 3))
sns.histplot(data=experiments_summary_train[experiments_summary_train['efficiency'] > 0], x='efficiency', kde=True)
plt.show()
fig.clf()

In [ ]:
for q in range(100):
    if np.percentile(experiments_summary_train['efficiency'], q) >= 350:
        print(q)
        break

Los dos conjuntos siguen la misma distribución para las variables como es de esperar, además de contar con los outliers comentados en el enunciado del ejercicio correspondientes al valor 0 para las variables gravity y wind_power.

Para la variable total_timesteps, podemos ver que aunque el 75% de las entradas tiene de un valor aproximadamente menor a 240 existen entradas que se extienden hasta los 900. 

Para la variable total_timesteps podemos ver que el 75% de las series cuenta con un timestamp menor a 242. El resto de casos se tratan de outliers, es decir, experimentos para los que el aterrizaje ha necesitado más tiempo. Esta variable se trata de una distribución de cola larga sesgada a la derecha.

Para las variables gravity y wind_power siguen una distribución uniforme, a excepción de las entradas con valor 0 para estas variables ya que se tratan de valores no registrados por el sistema.

Para la variable efficiency se observa una distribución de cola larga muy sesgada a la izquierda. El 3% de las entradas cuenta con un valor negativo para efficiency, por lo que la mayoría de entradas cuentan con un valor positivo para la variable. Además, un 91% de estas se encuentran entre 350 y 425.6.

### Primera toma de contacto
Tiene pinta de ser un problema de aprendizaje por refuerzo (reinforcement learning), pero creo que también podría ser un problema de redes neuronales recurrentes.

#### Aprendizaje por refuerzo
El aprendizaje por refuerzo es una técnica de aprendizaje que consiste en una serie de iteraciones de entrenamiento, donde el objetivo es aprender a predecir el estado siguiente a partir del estado actual. https://es.wikipedia.org/wiki/Aprendizaje_por_refuerzo
![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Reinforcement_learning_diagram.svg/250px-Reinforcement_learning_diagram.svg.png "RL")


Pensandolo mejor, es un problema supervisado, por lo que no creo que sea un problema de aprendizaje por refuerzo, aunque sí parece que el "simulador" que genera estos experimentos internamente es una inteligencia artificial de Reinforcement Learning.
Nosotros lo que tenemos que hacer es aprender a predecir la valoración de todo el conjunto de acciones de cada experimento.

#### Idea de red neuronal recurrente
La idea de la red neuronal recurrente es que la red se comporte como una máquina de estados, y que cada vez que se ejecuta una acción, se guarda el estado actual en una matriz de estados, y se guarda la acción que se ha tomado en una matriz de acciones. Podríamos probar con una LSTM.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/LSTM_Cell.svg/300px-LSTM_Cell.svg.png "LSTM")

# Planteamiento como problema de aprendizaje por refuerzo (reinforcement learning)

Un problema de aprendizaje por refuerzo cuenta con los siguientes elementos:

- Estado: Situación del agente. En el problema se representa por las variables:
    - x_pos, y_pos
    - x_vel, y_vel
    - angle, angle_vel
    - leg1, leg2

- Acciones: Acciones que puede realizar el agente. En el problema se representan por las variables:
    - main_booster
    - lat_booster


- Entorno: Entorno en el que se encuentra el agente. En el problema se representa por las variables:
    - gravity
    - wind_power
    - turbulences

# Preprocessing

## Adding static fields to time series

In [ ]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def safe_open_w(path):
    mkdir_p(os.path.dirname(path))
    return open(path, 'w')

In [ ]:
train_series_path = 'data/train'
test_series_path = 'data/test'
train_experiments_summary_path = 'data/experiments_summary_train.xlsx'
test_experiments_summary_path = 'data/experiments_summary_test.xlsx'

In [ ]:
# Now that we have added the static variables (gravity, wind_power, turbulence_power)
# Create a numpy array of shape (samples, timesteps, features) from several csv files and assign each to the target variable (efficiency)
# - Load the data from the csv files
# - Create a numpy array of shape (samples, timesteps, features)
# - Assign each numpy array to a target variable
# - Return the target variable
def transform_data(series_path, experiments_summary_path, target_variable=None):
    experiments_summary = pd.read_excel(experiments_summary_path)
    experiments_summary = experiments_summary.set_index('filename')

    X = []
    y = []
    for filename in os.listdir(series_path):
        series_file = os.path.join(series_path, filename)
        experiment_series = pd.read_csv(series_file)
        
        experiment_summary = experiments_summary.loc[filename]
        
        # Preprocessing steps
        # - Add static fields
        for static_var in ['gravity', 'wind_power', 'turbulence_power']:
            experiment_series[static_var] = np.repeat(experiment_summary[static_var], experiment_summary['total_timesteps']+1)
        
        X.append(experiment_series.values)

        if target_variable is not None:
            y.append(experiment_summary[target_variable])
    
    if target_variable is not None:
        return np.array(X), np.array(y)
    else:
        return np.array(X)

train_X, train_y = transform_data(train_series_path, train_experiments_summary_path, 'efficiency')
test_X = transform_data(test_series_path, test_experiments_summary_path)

# Dump the data to a numpy files
np.save('data/train_X.npy', train_X)
np.save('data/train_y.npy', train_y)
np.save('data/test_X.npy', test_X)

In [ ]:
# Load the data from the numpy files and transform it to (samples, timesteps, features)
train_X, train_y = np.load('data/train_X.npy', allow_pickle=True), np.load('data/train_y.npy', allow_pickle=True)
test_X = np.load('data/test_X.npy', allow_pickle=True)

print(train_X.shape, train_y.shape)
print(train_X[0].shape, train_y[0].shape)
print(train_X[0], train_y[0])

In [ ]:
# Pad the data to make it of the same length (max of all samples)
def pad_data(X):
    max_length = max([x.shape[0] for x in X])
    X_padded = []
    for x in X:
        x_padded = np.zeros((max_length, x.shape[1]))
        x_padded[:x.shape[0], :] = x
        X_padded.append(x_padded)
    return np.array(X_padded)

train_X = pad_data(train_X)
test_X = pad_data(test_X)

print(train_X.shape, train_y.shape)
print(train_X[0].shape, train_y[0].shape)
print(test_X.shape)

In [ ]:
def reshape_data_lstm(X):
    return X.reshape((X.shape[0], X[0].shape[0], X[0].shape[1]))

train_X = reshape_data_lstm(train_X)
test_X = reshape_data_lstm(test_X)

print(train_X.shape, train_y.shape)
print(train_X[0].shape, train_y[0].shape)
print(test_X.shape)

## Scaling and imputing

# LSTM

Crearemos una red LSTM que reciba series temporales asociadas a cada uno de los experimentos. Estas series contarán con todos las variables asociadas al estado y las acciones del agente, además de introducir como nuevas variables las del entorno. Estas últimas tendrán el mismo valor para todos los pasos del experimento.

Dado que las distintas series son de longitud variable y se encuentran en distintos archivos csv, deberemos entrenar la red mediante mini batches de tamano 1 (cada uno de los experimentos de los archivos). Otra opción sería realizar padding sobre las series, pero al contar con un rango amplio de timesteps para las series además de contar con gran cantidad de valores atípicos no parece un enfoque adecuado.

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(16, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False)) # input_shape=(timesteps, features) return_sequences=False -> returns a single vector
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# Train the model with early stopping.
history = model.fit(train_X, train_y, epochs=50, batch_size=32, validation_split=0.2, verbose=2, shuffle=False, callbacks=[EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, restore_best_weights=True)])

# Plot the model's predictions
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
history.summary()